## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [75]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [76]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [78]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [79]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,0.261822,0.027321,0.001082,0.017344
1,0.226534,0.027110,0.026033,-0.006047
2,-0.222401,0.283597,0.298611,0.370957
3,-0.194337,-0.477472,0.050774,0.156190
4,-0.227077,0.125684,-0.356834,-0.339252


In [80]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USW00093861', u'USW00012832', u'USC00097276', u'USC00092736', u'USC00081544']


,0,1,2,3
station,,,,
USW00093861,0.261822,0.027321,0.001082,0.017344
USW00012832,0.226534,0.027110,0.026033,-0.006047
USC00097276,-0.222401,0.283597,0.298611,0.370957
USC00092736,-0.194337,-0.477472,0.050774,0.156190
USC00081544,-0.227077,0.125684,-0.356834,-0.339252


In [81]:
table=Eig.join(Stations,how='left')
table

,0,1,2,3,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
station,,,,,,,,,,,,
USW00093861,0.261822,0.027321,0.001082,0.017344,30.8333,-83.9833,86.0,A,THOMASVILLE WB CITY,NaN,NaN,NaN
USW00012832,0.226534,0.027110,0.026033,-0.006047,29.7333,-85.0333,5.8,L,APALACHICOLA AP,NaN,HCN,72220.0
USC00097276,-0.222401,0.283597,0.298611,0.370957,30.7836,-83.5692,56.4,A,QUITMAN 2 NW,NaN,HCN,NaN
USC00092736,-0.194337,-0.477472,0.050774,0.156190,31.0167,-84.8833,41.1,A,DONALSONVILLE 1 S,NaN,NaN,NaN
USC00081544,-0.227077,0.125684,-0.356834,-0.339252,30.7836,-85.4847,39.6,L,CHIPLEY,NaN,NaN,NaN
USC00085879,0.261822,0.027321,0.001082,0.017344,30.4922,-83.7833,29.9,L,MONTICELLO 5 SE,NaN,NaN,NaN
USC00092153,0.261822,0.027321,0.001082,0.017344,31.1681,-84.7664,46.6,A,COLQUITT 2 W,NaN,NaN,NaN
USC00095585,-0.291067,0.371569,-0.321387,0.139069,31.1167,-83.7667,91.4,A,MAUZY,NaN,NaN,NaN
USC00096087,-0.194437,-0.478311,0.049938,0.156751,31.1769,-83.7492,103.6,A,MOULTRIE 2 ESE,NaN,NaN,NaN


In [88]:
ind= [23, 22, 17, 19, 24, 18, 14,  4,  3,  5, 21, 15,  7, 16,  8, 11,  9,
        6,  1,  0, 20, 12, 13,  2, 10]
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
table


,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USW00093861,THOMASVILLE WB CITY,30.8333,-83.9833,86.0,0.261822,0.027321,0.001082,0.017344
USW00012832,APALACHICOLA AP,29.7333,-85.0333,5.8,0.226534,0.027110,0.026033,-0.006047
USC00097276,QUITMAN 2 NW,30.7836,-83.5692,56.4,-0.222401,0.283597,0.298611,0.370957
USC00092736,DONALSONVILLE 1 S,31.0167,-84.8833,41.1,-0.194337,-0.477472,0.050774,0.156190
USC00081544,CHIPLEY,30.7836,-85.4847,39.6,-0.227077,0.125684,-0.356834,-0.339252
USC00085879,MONTICELLO 5 SE,30.4922,-83.7833,29.9,0.261822,0.027321,0.001082,0.017344
USC00092153,COLQUITT 2 W,31.1681,-84.7664,46.6,0.261822,0.027321,0.001082,0.017344
USC00095585,MAUZY,31.1167,-83.7667,91.4,-0.291067,0.371569,-0.321387,0.139069
USC00096087,MOULTRIE 2 ESE,31.1769,-83.7492,103.6,-0.194437,-0.478311,0.049938,0.156751


In [83]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [84]:
min_lat,max_lat,min_long,max_long = box = (29, 32, -86, -83)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [ ]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

In [ ]:
order[0:23]
